In [20]:
import time

import numpy as np

import matplotlib.pyplot as plt

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.constants import R_earth
from poliastro.twobody.sampling import EpochsArray
from poliastro.twobody.propagation import CowellPropagator

from astropy import units as u
from astropy.time import Time, TimeDelta

from datetime import datetime
from osc2mean_dd import osc2mean
from sso_inc import inc_from_alt, raan_from_ltan, argl_difference
from perturbations import perturbations_coesa_J2_low, perturbations_coesa_J2_high


toc = time.time()

## Orbit
h = 400
alpha_wait = [0.93,0.93,0.93,0.95,0.95]
alpha_hd = [5,5,5,5,5,5]
start_date = datetime(2024,1,1,9,0,0)
ltan = 22.5

a = (R_earth.value/1000 + h) << u.km
ecc = 1e-6 << u.one
inc = inc_from_alt(h,ecc)[0] << u.deg   
raan = raan_from_ltan(Time(val=datetime.timestamp(start_date), format='unix'),ltan) << u.deg
argp = 1e-6 << u.deg
nu = 1e-6 << u.deg

assignment = [40, 60, 90, 110, 150]                                                       
delta_a = [1.1, 0.9, 1, 0.8, 1]
delta_nu = [-0.1,-0.1,-0.1,-0.1, -0.1]

epoch = Time(val=start_date.isoformat(), format='isot')
time_step = 900<<u.s
pred_days = 20

refsmalist = []
refsmalist_mean = []

trail_sats = 5

trailsmalist = [[] for i in range(trail_sats)]
trailsmalist_mean = [[] for i in range(trail_sats)]
elapsedsecs = [[] for i in range(trail_sats)]
angle_list = [[] for i in range(trail_sats)]

reference_orbit_list = []
trailing_orbit_list = []

start_date_prop = epoch

for sat in range(trail_sats):
    secs = 0

    reference_orbit = Orbit.from_classical(
        Earth,
        a,
        ecc,
        inc,
        raan,
        argp,
        nu,
        epoch
        )
    trailing_orbit = Orbit.from_classical(
        Earth,
        (a.value+delta_a[sat])<<u.km,
        ecc,
        inc,
        raan,
        argp,
        (nu.value+delta_nu[sat])<<u.deg,
        epoch
        )

    ref_mean = osc2mean(a.value, ecc.value, inc.value, raan.value, argp.value, nu.value)
    ref_mean_orbit = Orbit.from_classical(Earth, ref_mean[0]<<u.km, ref_mean[1]<<u.one, ref_mean[2]<<u.deg, ref_mean[3]<<u.deg, ref_mean[4]<<u.deg, nu, epoch)
    trail_mean = osc2mean(a.value+delta_a[sat], ecc.value, inc.value, raan.value, argp.value, nu.value+delta_nu[sat])
    trail_mean_orbit = Orbit.from_classical(Earth, trail_mean[0]<<u.km, trail_mean[1]<<u.one, trail_mean[2]<<u.deg, trail_mean[3]<<u.deg, trail_mean[4]<<u.deg, nu+(delta_nu[sat]<<u.deg), epoch)

    theta_err = (assignment[sat] - argl_difference(reference_orbit, trailing_orbit))%360
    
    tra_orb_pred = trailing_orbit.propagate(pred_days<<u.day, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_high))
    tra_pred_mean = osc2mean(tra_orb_pred.a.value, tra_orb_pred.ecc.value, tra_orb_pred.inc.to_value(u.deg), tra_orb_pred.raan.to_value(u.deg), tra_orb_pred.argp.to_value(u.deg), tra_orb_pred.nu.to_value(u.deg))
    tra_orb_pred_mean = Orbit.from_classical(Earth, tra_pred_mean[0]<<u.km, tra_pred_mean[1]<<u.one, tra_pred_mean[2]<<u.deg, tra_pred_mean[3]<<u.deg, tra_pred_mean[4]<<u.deg, tra_orb_pred.nu.to(u.deg), tra_orb_pred.epoch)
    
    theta_dot_dot = (tra_orb_pred_mean.n.to(u.deg/u.s) - trail_mean_orbit.n.to(u.deg/u.s)) / ((pred_days*60*60*24)<<u.s)
    t_hd = (ref_mean_orbit.n.to(u.deg/u.s) - trail_mean_orbit.n.to(u.deg/u.s)) / theta_dot_dot
    theta_hd = 0.5 * theta_dot_dot * t_hd**2
    t_wait = ((theta_err - theta_hd.value) / (ref_mean_orbit.n.to_value(u.deg/u.s) - trail_mean_orbit.n.to_value(u.deg/u.s)))

    t_wait = t_wait * alpha_wait[sat]     
    t_hd = t_hd * alpha_hd[sat]

    num_wait = int(t_wait / time_step.value)
    tofs_wait = TimeDelta(np.linspace(0, t_wait<<u.s, num=num_wait))
    reference_ephem = reference_orbit.to_ephem(EpochsArray(start_date_prop + tofs_wait, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
    trailing_ephem = trailing_orbit.to_ephem(EpochsArray(start_date_prop + tofs_wait, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))


    for t in range(len(tofs_wait)):

        secs += time_step.value

        ref_from_ephem = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[t])
        trail_from_ephem = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[t])

        if sat == trail_sats-1:
            refsmalist.append(ref_from_ephem.a.value)
        trailsmalist[sat].append(trail_from_ephem.a.value)

        ref_mean = osc2mean(
            ref_from_ephem.a.value,
            ref_from_ephem.ecc.value,
            ref_from_ephem.inc.to_value(u.deg),
            ref_from_ephem.raan.to_value(u.deg),
            ref_from_ephem.argp.to_value(u.deg),
            ref_from_ephem.nu.to_value(u.deg)

        )
        trail_mean = osc2mean(
            trail_from_ephem.a.value,
            trail_from_ephem.ecc.value,
            trail_from_ephem.inc.to_value(u.deg),
            trail_from_ephem.raan.to_value(u.deg),
            trail_from_ephem.argp.to_value(u.deg),
            trail_from_ephem.nu.to_value(u.deg)
        )

        if sat == trail_sats-1:
            refsmalist_mean.append(ref_mean[0])
        trailsmalist_mean[sat].append(trail_mean[0])
        
        angle_list[sat].append(argl_difference(ref_from_ephem, trail_from_ephem))

        elapsedsecs[sat].append(secs)
    
    reference_orbit = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[-1])
    trailing_orbit = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[-1])


    num_hd = int(t_hd.value / time_step.value)
    tofs_hd = TimeDelta(np.linspace(0, t_hd, num=num_hd))

    reference_ephem = reference_orbit.to_ephem(EpochsArray(reference_ephem.epochs[-1] + tofs_hd, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
    trailing_ephem = trailing_orbit.to_ephem(EpochsArray(trailing_ephem.epochs[-1] + tofs_hd, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_high)))


    for t in range(len(tofs_hd)):

        secs += time_step.value

        ref_from_ephem = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[t])
        trail_from_ephem = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[t])

        if sat == trail_sats-1:
            refsmalist.append(ref_from_ephem.a.value)
        trailsmalist[sat].append(trail_from_ephem.a.value)

        ref_mean = osc2mean(
            ref_from_ephem.a.value,
            ref_from_ephem.ecc.value,
            ref_from_ephem.inc.to_value(u.deg),
            ref_from_ephem.raan.to_value(u.deg),
            ref_from_ephem.argp.to_value(u.deg),
            ref_from_ephem.nu.to_value(u.deg)

        )
        trail_mean = osc2mean(
            trail_from_ephem.a.value,
            trail_from_ephem.ecc.value,
            trail_from_ephem.inc.to_value(u.deg),
            trail_from_ephem.raan.to_value(u.deg),
            trail_from_ephem.argp.to_value(u.deg),
            trail_from_ephem.nu.to_value(u.deg)
        )

        if sat == trail_sats-1:
            refsmalist_mean.append(ref_mean[0])
        trailsmalist_mean[sat].append(trail_mean[0])
        
        angle_list[sat].append(argl_difference(ref_from_ephem, trail_from_ephem))

        elapsedsecs[sat].append(secs)


###
    reference_orbit = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[-1])
    trailing_orbit = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[-1])

    ref_mean_orbit = Orbit.from_classical(Earth, ref_mean[0]<<u.km, ref_mean[1]<<u.one, ref_mean[2]<<u.deg, ref_mean[3]<<u.deg, ref_mean[4]<<u.deg, ref_from_ephem.nu.to(u.deg), ref_from_ephem.epoch)
    trail_mean_orbit = Orbit.from_classical(Earth, trail_mean[0]<<u.km, trail_mean[1]<<u.one, trail_mean[2]<<u.deg, trail_mean[3]<<u.deg, trail_mean[4]<<u.deg, trail_from_ephem.nu.to(u.deg), ref_from_ephem.epoch)


    ref_mean_orbit = Orbit.from_classical(
                                          Earth, 
                                          ref_mean[0]<<u.km, 
                                          ref_mean[1]<<u.one, 
                                          ref_mean[2]<<u.deg, 
                                          ref_mean[3]<<u.deg, 
                                          ref_mean[4]<<u.deg, 
                                          reference_orbit.nu.to(u.deg), 
                                          reference_orbit.epoch
                                          )

    trail_mean_orbit = Orbit.from_classical(
                                            Earth, 
                                            trail_mean[0]<<u.km, 
                                            trail_mean[1]<<u.one, 
                                            trail_mean[2]<<u.deg, 
                                            trail_mean[3]<<u.deg, 
                                            trail_mean[4]<<u.deg, 
                                            trailing_orbit.nu.to(u.deg), 
                                            trailing_orbit.epoch
                                            )
    

    t_prop = (60*60*24*7*2)<<u.s
    num_prop = int(t_prop.value / time_step.value)
    tofs_prop = TimeDelta(np.linspace(0, t_prop, num=num_prop))

    reference_ephem = reference_orbit.to_ephem(EpochsArray(reference_orbit.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
    trailing_ephem = trailing_orbit.to_ephem(EpochsArray(trailing_orbit.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))

    for t in range(len(tofs_prop)):

        secs += time_step.value

        ref_from_ephem = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[t])
        trail_from_ephem = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[t])

        if sat == trail_sats-1:
            refsmalist.append(ref_from_ephem.a.value)
        trailsmalist[sat].append(trail_from_ephem.a.value)

        ref_mean = osc2mean(
            ref_from_ephem.a.value,
            ref_from_ephem.ecc.value,
            ref_from_ephem.inc.to_value(u.deg),
            ref_from_ephem.raan.to_value(u.deg),
            ref_from_ephem.argp.to_value(u.deg),
            ref_from_ephem.nu.to_value(u.deg)

        )
        trail_mean = osc2mean(
            trail_from_ephem.a.value,
            trail_from_ephem.ecc.value,
            trail_from_ephem.inc.to_value(u.deg),
            trail_from_ephem.raan.to_value(u.deg),
            trail_from_ephem.argp.to_value(u.deg),
            trail_from_ephem.nu.to_value(u.deg)
        )

        if sat == trail_sats-1:
            refsmalist_mean.append(ref_mean[0])
        trailsmalist_mean[sat].append(trail_mean[0])
        
        angle_list[sat].append(argl_difference(ref_from_ephem, trail_from_ephem))

        elapsedsecs[sat].append(secs)

## time-equalizer    
# min_time = min(len(elapsedsecs[0]), len(elapsedsecs[1]), len(elapsedsecs[2]))    

# for i in range(trail_sats):
#     if len(elapsedsecs[i]) == min_time:





# t_prop = (elapsedsecs[short_time-1][-1] - elapsedsecs[short_time][-1])<<u.s
# num_prop = int(t_prop.value / time_step.value)
# tofs_prop = TimeDelta(np.linspace(0, t_prop, num=num_prop))

# reference_ephem = reference_orbit_short.to_ephem(EpochsArray(reference_orbit_short.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
# trailing_ephem = trailing_orbit_short.to_ephem(EpochsArray(trailing_orbit_short.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))

# for t in range(len(tofs_prop)):

#     ref_from_ephem = Orbit.from_ephem(Earth, reference_ephem, reference_ephem.epochs[t])
#     trail_from_ephem = Orbit.from_ephem(Earth, trailing_ephem, trailing_ephem.epochs[t])

#     trailsmalist[short_time].append(trail_from_ephem.a.value)

#     trail_mean = osc2mean(
#         trail_from_ephem.a.value,
#         trail_from_ephem.ecc.value,
#         trail_from_ephem.inc.to_value(u.deg),
#         trail_from_ephem.raan.to_value(u.deg),
#         trail_from_ephem.argp.to_value(u.deg),
#         trail_from_ephem.nu.to_value(u.deg)
#     )

#     trailsmalist_mean[short_time].append(trail_mean[0])

#     angle_list[short_time].append(argl_difference(ref_from_ephem, trail_from_ephem))


# trailing_orbit_short = trail_from_ephem

# print(ref_from_ephem.epoch)
# print(trailing_orbit_short.epoch)
# print(trailing_orbit_long.epoch)


# t_prop = (60*60*24*7*4)<<u.s
# num_prop = int(t_prop.value / time_step.value)
# tofs_prop = TimeDelta(np.linspace(0, t_prop, num=num_prop))

# reference_ephem_short = ref_from_ephem.to_ephem(EpochsArray(reference_ephem.epochs[-1] + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
# reference_ephem_long = reference_orbit_long.to_ephem(EpochsArray(reference_orbit_long.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
# trailing_ephem_short = trailing_orbit_short.to_ephem(EpochsArray(trailing_orbit_short.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))
# trailing_ephem_long = trailing_orbit_long.to_ephem(EpochsArray(trailing_orbit_long.epoch + tofs_prop, method=CowellPropagator(rtol=1e-5, f=perturbations_coesa_J2_low)))

# for t in range(len(tofs_prop)):

#     secs += time_step.value

#     ref_from_ephem_short = Orbit.from_ephem(Earth, reference_ephem_short, reference_ephem_short.epochs[t])
#     ref_from_ephem_long = Orbit.from_ephem(Earth, reference_ephem_long, reference_ephem_long.epochs[t])
#     trail_from_ephem_short = Orbit.from_ephem(Earth, trailing_ephem_short, trailing_ephem_short.epochs[t])
#     trail_from_ephem_long = Orbit.from_ephem(Earth, trailing_ephem_long, trailing_ephem_long.epochs[t])


#     refsmalist.append(ref_from_ephem.a.value)

#     ref_mean = osc2mean(
#         ref_from_ephem_short.a.value,
#         ref_from_ephem_short.ecc.value,
#         ref_from_ephem_short.inc.to_value(u.deg),
#         ref_from_ephem_short.raan.to_value(u.deg),
#         ref_from_ephem_short.argp.to_value(u.deg),
#         ref_from_ephem_short.nu.to_value(u.deg)

#     )
#     trail_mean_short = osc2mean(
#         trail_from_ephem_short.a.value,
#         trail_from_ephem_short.ecc.value,
#         trail_from_ephem_short.inc.to_value(u.deg),
#         trail_from_ephem_short.raan.to_value(u.deg),
#         trail_from_ephem_short.argp.to_value(u.deg),
#         trail_from_ephem_short.nu.to_value(u.deg)
#     )
#     trail_mean_long = osc2mean(
#         trail_from_ephem_long.a.value,
#         trail_from_ephem_long.ecc.value,
#         trail_from_ephem_long.inc.to_value(u.deg),
#         trail_from_ephem_long.raan.to_value(u.deg),
#         trail_from_ephem_long.argp.to_value(u.deg),
#         trail_from_ephem_long.nu.to_value(u.deg)
#     )

#     refsmalist_mean.append(ref_mean[0])
#     trailsmalist_mean[short_time].append(trail_mean_short[0])
#     trailsmalist_mean[short_time-1].append(trail_mean_long[0])
    
#     angle_list[short_time].append(argl_difference(ref_from_ephem_short, trail_from_ephem_short))
#     angle_list[short_time-1].append(argl_difference(ref_from_ephem_long, trail_from_ephem_long))

#     elapsedsecs[1].append(secs)


# elapsedsecs[0] = elapsedsecs[1]


elapsed_days = [[] for i in range(trail_sats)]

for sat in range(trail_sats):
    for sec in range(len(elapsedsecs[sat])):
        elapsed_days[sat].append(elapsedsecs[sat][sec]/(60*60*24))

ref_mean_altitudes = []
for sma in range(len(refsmalist_mean)):
    ref_mean_altitudes.append(refsmalist_mean[sma] - Earth.R_mean.to_value(u.km))

trail_mean_altitudes = [[] for i in range(trail_sats)]
for sat in range(trail_sats):
    for sma in range(len(trailsmalist_mean[sat])):
        trail_mean_altitudes[sat].append(trailsmalist_mean[sat][sma] - Earth.R_mean.to_value(u.km))


fig, ax = plt.subplots(1, 2, figsize=(22,9), squeeze=False) 

for i in range(trail_sats):
    ax[0,0].plot(elapsed_days[i],angle_list[i])
    # ax[0,0].plot(elapsedsecs[1],angle_list[1])
    # ax[0,0].plot(elapsedsecs[2],angle_list[2])
    ax[0,0].axhline(assignment[i],linestyle='--',color='red',label = f'Assigned Slot at {assignment[i]}deg')
    # ax[0,0].axhline(assignment[1],linestyle='--',color='red',label = f'Assigned Slot at {assignment[1]}deg')
    # ax[0,0].axhline(assignment[2],linestyle='--',color='red',label = f'Assigned Slot at {assignment[2]}deg')
ax[0,0].legend(loc = 'upper left')
ax[0,0].set_title('Angle Between Satellites')
ax[0,0].set_xlabel('Days')
ax[0,0].set_ylabel('Degrees')

for i in range(trail_sats):
    ax[0,1].plot(elapsed_days[i],trail_mean_altitudes[i],label='Trail')
    # ax[0,1].plot(elapsedsecs[1],trail_mean_altitudes[1],label='Trail')
    # ax[0,1].plot(elapsedsecs[2],trail_mean_altitudes[2],label='Trail')
ax[0,1].plot(elapsed_days[-1],ref_mean_altitudes,label='Ref')
ax[0,1].set_title('Ref vs Trail Mean Altitude')
ax[0,1].set_xlabel('Days')
ax[0,1].set_ylabel('Km')

tic = time.time()
print(f'Timestep {time_step:.4f}s')
print(f'Run time {tic-toc:.2f}s/{(tic-toc)/60:.2f}m')
plt.show()